# Historical Covid API Data

# PPP Loan Data > 150k

In [2]:
import pandas as pd
import pickle
import geocoder as geo
import folium as f
import requests
import re

In [29]:
df_loan = pd.read_csv('data/ppp_150.csv')

cols = ['LoanRange', 'BusinessName', 'Address', 'City', 'State', 'Zip', 'NAICSCode', 'BusinessType', 'JobsRetained']

In [30]:
df_loan_strip = df_loan[cols]

In [28]:
df_loan_strip.size

5950962

In [8]:
df_loan_strip.to_json('data/ppp_loan_strip.json')

In [1]:
df_loan_strip.to_csv('data/ppp_loan_strip.csv')

NameError: name 'df_loan_strip' is not defined

In [38]:
df_loan_strip = pd.read_csv("data/ppp_loan_strip.csv", index_col=0)

In [46]:
a = df_loan_strip[df_loan_strip['City'] == "CHRISTIANSBURG"]

In [47]:
a[a['State'] == 'VA']

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,JobsRetained
611483,a $5-10 million,GILLMANN SERVICES INC,909 CAMBRIA ST,CHRISTIANSBURG,VA,24073.0,561320.0,Corporation,408.0
611530,a $5-10 million,SHELOR MOTOR MILE INC,2257 ROANOKE ST,CHRISTIANSBURG,VA,24073.0,441110.0,Corporation,0.0
611679,b $2-5 million,COMMUNITY HOUSING PARTNERS CORP,"448 Depot St, NE",CHRISTIANSBURG,VA,24073.0,624229.0,Non-Profit Organization,340.0
612107,b $2-5 million,"TWENTY 20 MANAGEMENT, INC.",105 Suite 400 Arbor Dr.,CHRISTIANSBURG,VA,24073.0,623110.0,Corporation,0.0
613415,c $1-2 million,"VIRGINIA COMMUNITY CAPITAL, INC.",110 Peppers Ferry Road NW,CHRISTIANSBURG,VA,24073.0,522390.0,Non-Profit Organization,65.0
614024,"d $350,000-1 million",BILL'S USED AUTO PARTS,1415 RADFORD ROAD,CHRISTIANSBURG,VA,24073.0,423140.0,Subchapter S Corporation,48.0
614075,"d $350,000-1 million","BLUE RIDGE HEATING &AMP; AIR, INC.",925 Cambria St NE,CHRISTIANSBURG,VA,24073.0,238220.0,Corporation,40.0
614836,"d $350,000-1 million",DUNCAN ENTERPRISES COMPANY VA,2040 ROANOKE ST,CHRISTIANSBURG,VA,24073.0,441110.0,Corporation,47.0
614837,"d $350,000-1 million",DUNCAN IMPORTS INC,2050 Roanoke St,CHRISTIANSBURG,VA,24073.0,441110.0,Corporation,34.0
615118,"d $350,000-1 million","FIRST TEAM NISSAN CHRISTAINSBURG, LLC",2130 N. FRANKLIN ST,CHRISTIANSBURG,VA,24073.0,441110.0,Limited Liability Company(LLC),0.0


In [40]:
df_loan_strip

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,JobsRetained
0,a $5-10 million,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,99723.0,813920.0,Non-Profit Organization,295.0
1,a $5-10 million,CRUZ CONSTRUCTION INC,7000 East Palmer Wasilla Hwy,PALMER,AK,99645.0,238190.0,Subchapter S Corporation,215.0
2,a $5-10 million,"I. C. E. SERVICES, INC",2606 C Street,ANCHORAGE,AK,99503.0,722310.0,Corporation,367.0
3,a $5-10 million,KATMAI HEALTH SERVICES LLC,"11001 O'MALLEY CENTRE DRIVE, SUITE 204",ANCHORAGE,AK,99515.0,621111.0,Limited Liability Company(LLC),0.0
4,a $5-10 million,MATANUSKA TELEPHONE ASSOCIATION,1740 S. CHUGACH ST,PALMER,AK,99645.0,517311.0,Cooperative,267.0
...,...,...,...,...,...,...,...,...,...
661213,"e $150,000-350,000",NOT AVAILABLE,NaN,NaN,XX,NaN,339114.0,NaN,17.0
661214,"e $150,000-350,000",NOT AVAILABLE,NaN,NaN,XX,NaN,339114.0,NaN,16.0
661215,"e $150,000-350,000",NOT AVAILABLE,NaN,NaN,XX,NaN,339114.0,NaN,14.0
661216,"e $150,000-350,000",OTTAWA PRODUCTS CO INC,NaN,NaN,XX,NaN,332510.0,NaN,NaN


In [20]:
df_loan_strip.sort_values(by='LoanRange')

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,JobsRetained
0,a $5-10 million,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,99723.0,813920.0,Non-Profit Organization,295.0
492060,a $5-10 million,THIRSTY LION EXPANSION LLC,829 SE 9th Ave Ste 202,PORTLAND,OR,97214.0,722511.0,Limited Liability Company(LLC),500.0
492059,a $5-10 million,"THE CORVALLIS CLINIC, P.C.",444 NW Elks Drive,CORVALLIS,OR,97330.0,621111.0,Corporation,500.0
492058,a $5-10 million,"TEC OF CALIFORNIA, INC.",750 NE Columbia Blvd.,PORTLAND,OR,97211.0,441228.0,Corporation,480.0
492057,a $5-10 million,SWANSON GROUP INC,2635 OLD HWY 99 S,ROSEBURG,OR,97471.0,321113.0,Corporation,NaN
...,...,...,...,...,...,...,...,...,...
138998,"e $150,000-350,000",HARRISON HOUSE COMMUNITY PROGRAMS INC,1415 PULASKI HWY,NEWARK,DE,19702.0,813410.0,Limited Liability Company(LLC),28.0
138999,"e $150,000-350,000",HARVARD ENVIRONMENTAL INC,760 PULASKI HIGHWAY,BEAR,DE,19701.0,541620.0,Subchapter S Corporation,NaN
139000,"e $150,000-350,000",HARVEST COMMUNITY DEVELOPMENT CORPORATION,2205 Lancaster Ave,WILMINGTON,DE,19805.0,611110.0,Professional Association,0.0
138990,"e $150,000-350,000","HABITAT FOR HUMANITY OF NEW CASTLE COUNTY, INC.",1920 Hutton Street,WILMINGTON,DE,19802.0,236115.0,Non-Profit Organization,32.0


# Create Loan Type Category Column

In [46]:
df_loan_strip['LoanCat'] = df_loan_strip.apply(lambda row: re.search(r"[abcde]", row['LoanRange'])[0], axis=1)

<ipython-input-46-c732ee95b3bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loan_strip['LoanCat'] = df_loan_strip.apply(lambda row: re.search(r"[abcde]", row['LoanRange'])[0], axis=1)


In [47]:
del df_loan_strip['LoanRange']

In [75]:
type(df_loan_strip.iloc[661217].Address)

float

In [105]:
na_addr = df_loan_strip[df_loan_strip['Address'].isna()]
na_city = df_loan_strip[df_loan_strip['City'].isna()]
na_zip = df_loan_strip[df_loan_strip['Zip'].isna()]
na_jobs = df_loan_strip[df_loan_strip['JobsRetained'].isna()]
zero_jobs = df_loan_strip[df_loan_strip['JobsRetained'] == 0]

In [106]:
na_city.shape, na_addr.shape, na_zip.shape, na_jobs.shape, zero_jobs.shape

((15, 9), (17, 9), (16, 9), (40506, 9), (48922, 9))

In [111]:
loan_type_list = ['a', 'b', 'c', 'd', 'e']

for l_type in loan_type_list:
    df_loan_strip[df_loan_strip['LoanCat'] == l_type].to_csv('data/loan_strip_cat' + l_type.capitalize() + '.csv')

### 13% of borrowers reported zero jobs retained

In [112]:
df_loan[df_loan['JobsRetained'].isna()].size/df_loan_strip.size

0.10890608946923204

In [4]:
df_loan_strip[df_loan_strip['JobsRetained'].isna()].size/df_loan_strip.size

0.06125967532644302

In [5]:
df_loan_strip[df_loan_strip['JobsRetained'] == 0].size/df_loan_strip.size

0.07398770148423062

### Remove null and invalid states

In [15]:
df_addr = df_loan[['LoanRange', 'Address', 'City', 'State', 'Zip']].dropna()

In [16]:
df_addr[df_addr['State'] == 'XX']

,LoanRange,Address,City,State,Zip
661203,"d $350,000-1 million",1850 BOYER AVE E,SEATTLE,XX,98112.0
661204,"d $350,000-1 million",1661 MARTIN RANCH RD,SAN BERNARDINO,XX,92407.0
661205,"d $350,000-1 million",7684 Southrail Road,NORTH CHARLESTON,XX,29420.0
661206,"d $350,000-1 million",2410 Highway 15 South,SUMTER,XX,29150.0
661209,"e $150,000-350,000",501 State Road 13,SAINT JOHNS,XX,32259.0


In [17]:
df_addr = df_addr[df_addr['State'] != 'XX']

In [18]:
len(df_addr)

661195

# Geocoder lib

In [5]:
a_addr = pd.read_csv('data/loan_strip_catA.csv', index_col=0)

In [13]:
len(a_addr)

4840

In [12]:
a_addr.head()

,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,JobsRetained,LoanCat
0,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,99723.0,813920.0,Non-Profit Organization,295.0,a
1,CRUZ CONSTRUCTION INC,7000 East Palmer Wasilla Hwy,PALMER,AK,99645.0,238190.0,Subchapter S Corporation,215.0,a
2,"I. C. E. SERVICES, INC",2606 C Street,ANCHORAGE,AK,99503.0,722310.0,Corporation,367.0,a
3,KATMAI HEALTH SERVICES LLC,"11001 O'MALLEY CENTRE DRIVE, SUITE 204",ANCHORAGE,AK,99515.0,621111.0,Limited Liability Company(LLC),0.0,a
4,MATANUSKA TELEPHONE ASSOCIATION,1740 S. CHUGACH ST,PALMER,AK,99645.0,517311.0,Cooperative,267.0,a


def construct_full_addr(row):
    a_addr['full_addr'] = ((row['Address'].split(',')[0] + ', ' 
                            + row['City'] + ', ' + row['State']))

# Add full address to DF

### Construct Full Addresses

`.split(',')` removes address appendices such as "Suite A", "Room 17", etc.

In [7]:
a_addr['full_addr'] = a_addr.apply(lambda row: (row['Address'].split(',')[0] + ', ' 
                            + row['City'] + ', ' + row['State']), axis=1)

### Get latitude and longitude for addresses of recipients of A-class loans ($5-10 million)

In [46]:
with requests.Session() as session:
    a_addr['latlng'] = a_addr.apply(lambda row: geo.osm(row['full_addr'], session=session).latlng, axis=1)

### Get latlng for B-class loans

In [6]:
b_addr = pd.read_csv('data/loan_strip_catB.csv', index_col=0)

In [7]:
len(b_addr)

24838

In [12]:
b_addr['full_addr'] = b_addr.apply(lambda row: (row['Address'].split(',')[0] + ', ' 
                            + row['City'] + ', ' + row['State']), axis=1)

In [13]:
b_addr.head()

,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,JobsRetained,LoanCat,full_addr
12,"ADVANCED SUPPLY CHAIN INTERNATIONAL, LLC",2525 GAMBELL ST Ste 405,ANCHORAGE,AK,99503.0,493190.0,Limited Liability Company(LLC),126.0,b,"2525 GAMBELL ST Ste 405, ANCHORAGE, AK"
13,AHTNA CONSTRUCTION &AMP; PRIMARY PRODUCTS COMP...,110 W 38th Ave Ste 100,ANCHORAGE,AK,99503.0,237310.0,NaN,135.0,b,"110 W 38th Ave Ste 100, ANCHORAGE, AK"
14,AHTNA ENVIRONMENTAL INC,110 W 38th Ave Ste 200B,ANCHORAGE,AK,99503.0,562910.0,Corporation,180.0,b,"110 W 38th Ave Ste 200B, ANCHORAGE, AK"
15,AHTNA PROFESSIONAL SERVICES INC,110 W 38th Ave Ste 200F,ANCHORAGE,AK,99503.0,541513.0,Corporation,216.0,b,"110 W 38th Ave Ste 200F, ANCHORAGE, AK"
16,AHTNA SUPPORT AND TRAINING SERVICES LLC,110 W 38th Ave Ste 100,ANCHORAGE,AK,99503.0,541690.0,NaN,484.0,b,"110 W 38th Ave Ste 100, ANCHORAGE, AK"


In [23]:
with requests.Session() as session:
    b_addr['latlng'] = b_addr.apply(lambda row: geo.osm(row['full_addr'], session=session).latlng, axis=1)

In [24]:
b_addr.head()

,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,JobsRetained,LoanCat,full_addr,latlng
12,"ADVANCED SUPPLY CHAIN INTERNATIONAL, LLC",2525 GAMBELL ST Ste 405,ANCHORAGE,AK,99503.0,493190.0,Limited Liability Company(LLC),126.0,b,"2525 GAMBELL ST Ste 405, ANCHORAGE, AK","[61.197596747753884, -149.8701726300791]"
13,AHTNA CONSTRUCTION &AMP; PRIMARY PRODUCTS COMP...,110 W 38th Ave Ste 100,ANCHORAGE,AK,99503.0,237310.0,NaN,135.0,b,"110 W 38th Ave Ste 100, ANCHORAGE, AK","[61.18608644671806, -149.88438297129954]"
14,AHTNA ENVIRONMENTAL INC,110 W 38th Ave Ste 200B,ANCHORAGE,AK,99503.0,562910.0,Corporation,180.0,b,"110 W 38th Ave Ste 200B, ANCHORAGE, AK",None
15,AHTNA PROFESSIONAL SERVICES INC,110 W 38th Ave Ste 200F,ANCHORAGE,AK,99503.0,541513.0,Corporation,216.0,b,"110 W 38th Ave Ste 200F, ANCHORAGE, AK",None
16,AHTNA SUPPORT AND TRAINING SERVICES LLC,110 W 38th Ave Ste 100,ANCHORAGE,AK,99503.0,541690.0,NaN,484.0,b,"110 W 38th Ave Ste 100, ANCHORAGE, AK","[61.18608644671806, -149.88438297129954]"


In [25]:
pickle.dump(b_addr, open("static/b_addr.pkd", "wb"))

In [28]:
b_addr[b_addr['latlng'].isna()]

,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,JobsRetained,LoanCat,full_addr,latlng
14,AHTNA ENVIRONMENTAL INC,110 W 38th Ave Ste 200B,ANCHORAGE,AK,99503.0,562910.0,Corporation,180.0,b,"110 W 38th Ave Ste 200B, ANCHORAGE, AK",None
15,AHTNA PROFESSIONAL SERVICES INC,110 W 38th Ave Ste 200F,ANCHORAGE,AK,99503.0,541513.0,Corporation,216.0,b,"110 W 38th Ave Ste 200F, ANCHORAGE, AK",None
19,"ALASKA HEART INSTITUTE, LLC",3841 Piper Street Suite T-100,ANCHORAGE,AK,99508.0,621111.0,Subchapter S Corporation,240.0,b,"3841 Piper Street Suite T-100, ANCHORAGE, AK",None
26,ANCHORAGE FRACTURE & ORTHOPEDIC CLINIC,3831 Piper Street Suite 220,ANCHORAGE,AK,99508.0,621111.0,Corporation,117.0,b,"3831 Piper Street Suite 220, ANCHORAGE, AK",None
29,"BERING AIR, INC.",PO Box 1650,NOME,AK,99762.0,481211.0,Subchapter S Corporation,184.0,b,"PO Box 1650, NOME, AK",None
...,...,...,...,...,...,...,...,...,...,...,...
659860,NORTHERN ARAPAHO ENTERPRISE 2,PO Box 1989,RIVERTON,WY,82501.0,921150.0,Corporation,240.0,b,"PO Box 1989, RIVERTON, WY",None
659862,"R&R SERVICES, INC.",57 BILL HALL RD,DOUGLAS,WY,82633.0,213112.0,Corporation,137.0,b,"57 BILL HALL RD, DOUGLAS, WY",None
659868,WESTERN ECOSYSTEMS TECHNOLOGY INC.,415 W 17th Street Suite 200,CHEYENNE,WY,82001.0,541990.0,Corporation,302.0,b,"415 W 17th Street Suite 200, CHEYENNE, WY",None
659869,WIND RIVER CASINO,180 Rd Wolf Pl,RIVERTON,WY,82501.0,921150.0,Corporation,245.0,b,"180 Rd Wolf Pl, RIVERTON, WY",None


### Pickling

In [48]:
pickle.dump(a_addr, open("a_adrr.pkd", "wb"))

In [14]:
a_addr = pickle.load(open("a_addr.pkd", "rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'a_addr.pkd'

### Useful function for checking actual size of an object

https://stackoverflow.com/questions/449560/how-do-i-determine-the-size-of-an-object-in-python

In [61]:
import sys
from types import ModuleType, FunctionType
from gc import get_referents

# Custom objects know their class.
# Function objects seem to know way too much, including modules.
# Exclude modules as well.
BLACKLIST = type, ModuleType, FunctionType


def getsize(obj):
    """sum size of object & members."""
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size

# Map

In [70]:
# create map
m = f.Map(location=[44.58, -103.46], zoom_start=3)

# global tooltip
tooltip = 'Click for more info'

# generate map
m.save('map.html')

In [71]:
def add_aClass_loans(latlng):
    f.CircleMarker(
        location=latlng,
        radius=2,  # use for loan size? scale to size of loan? normalize these?
        popup="$5-10 Million",  # loan value
        color='blue',  # color according to some histogram?
        fill=False,
        fill_color='blue'
    ).add_to(m)

In [72]:
def add_loans_to_map():
    for i in a_latlng.itertuples():
        if i.latlng != None:
            add_aClass_loans(i.latlng)
#             print(i.latlng)

In [35]:
a_latlng[a_latlng['latlng'].isna()]

,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,JobsRetained,LoanCat,full_addr,latlng
5,"NANA WORLEY, LLC","3700 Centerpoint Drive, 7th floor",ANCHORAGE,AK,99503.0,541330.0,Limited Liability Company(LLC),231.0,a,"3700 Centerpoint Drive, ANCHORAGE, AK",None
11,"YULISTA TACTICAL, LLC",5015 BUSINESS PARK BLVD Suite 3000,ANCHORAGE,AK,99503.0,541614.0,Limited Liability Company(LLC),220.0,a,"5015 BUSINESS PARK BLVD Suite 3000, ANCHORAGE, AK",None
13,"ALLIANCE HR, INC",307 CLINTON AVE suite 200,HUNTSVILLE,AL,35801.0,561311.0,Corporation,443.0,a,"307 CLINTON AVE suite 200, HUNTSVILLE, AL",None
19,ATS OPERATING LLC,1900 CRESTWOOD BLVD SUITE 302,IRONDALE,AL,35210.0,453310.0,Limited Liability Company(LLC),250.0,a,"1900 CRESTWOOD BLVD SUITE 302, IRONDALE, AL",None
22,BEVILACQUA RESEARCH CORPORATION,4901 CORPORATE DR Ste B,HUNTSVILLE,AL,35805.0,541330.0,Corporation,343.0,a,"4901 CORPORATE DR Ste B, HUNTSVILLE, AL",None
...,...,...,...,...,...,...,...,...,...,...,...
4826,"LEXINGTON COAL COMPANY, INC",1051 Main St,MILTON,WV,25541.0,213113.0,Limited Liability Company(LLC),139.0,a,"1051 Main St, MILTON, WV",None
4828,NOBORNE PRESCHOOL AND DAYCARE CENTER,200 KING ST,MARTINSBURG,WV,25401.0,624410.0,Non-Profit Organization,17.0,a,"200 KING ST, MARTINSBURG, WV",None
4831,TRITON CONSTRUCTION INC,1944 WINFIELD RD,SAINT ALBANS,WV,25177.0,237310.0,Subchapter S Corporation,264.0,a,"1944 WINFIELD RD, SAINT ALBANS, WV",None
4832,"CYCLONE DRILLING, INC.",5800 Mohan Road PO Box 908,GILLETTE,WY,82717.0,213112.0,Corporation,364.0,a,"5800 Mohan Road PO Box 908, GILLETTE, WY",None


In [73]:
add_loans_to_map()

In [74]:
m.save('map.html')

# Separate by loan range

In [75]:
dollars = re.compile(r"(?P<amt>\d+)|(?P<mag>million|thousand)")

In [84]:
re.search(dollars, loan)

<re.Match object; span=(3, 4), match='5'>

In [94]:
lr = loan_range = re.findall(dollars, loan)
lr = str(1.*(int(lr[1][0]) + int(lr[0][0]))/2) + ' ' + lr[2][1]

In [104]:
a = re.search(lr, r"\f")

In [106]:
lr

'7.5 million'

In [60]:
lat, long = geo[0]['geometry']['location']['lat'], geo[0]['geometry']['location']['lng']
m = f.Map([lat, long], zoom_start=12)

In [109]:
f.CircleMarker(
    location=[lat, long],
    radius=7.5,  # use for loan size? scale to size of loan? normalize these?
    popup=str(lr),  # loan value
    color='blue',  # color according to some histogram?
    fill=True,
    fill_color='blue'
).add_to(m)

In [110]:
m.save('ppp_test_map.html')